In [1]:
#Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload


In [2]:
monthly_riders_since_2008 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')

#No missing values
monthly_riders_since_2008.dropna(inplace=True)

#Filter to only include Subway and no other MTA agency
monthly_riders_since_2008 = monthly_riders_since_2008[monthly_riders_since_2008['Agency'] == 'Subway'] 

#Rename "Month" column into Date
monthly_riders_since_2008.rename(columns = {'Month':'Date'}, inplace = True)

#Polish Date column and conver to datetime
monthly_riders_since_2008['Date'] = monthly_riders_since_2008['Date'].str.replace('/01/', '/', regex=True)
monthly_riders_since_2008['Date'] = pd.to_datetime(monthly_riders_since_2008['Date'], format='%m/%Y')
#monthly_riders_since_2008['Date'] = monthly_riders_since_2008['Date'].dt.strftime('%m/%Y')

#Break out date into coumns for date, month, and year
monthly_riders_since_2008['Month'] = monthly_riders_since_2008['Date'].dt.month
monthly_riders_since_2008['Year'] = monthly_riders_since_2008['Date'].dt.year

#Drop Date column now
monthly_riders_since_2008.drop(['Date', 'Agency'], inplace=True, axis=1)

#Filter out the year 2024
monthly_riders_since_2008 = monthly_riders_since_2008[monthly_riders_since_2008['Year'] < 2024] 

#Groupby Year and look at mean ridership
monthly_riders_since_2008['Mean Ridership By Year'] = monthly_riders_since_2008.groupby('Year')['Ridership'].transform(lambda x: x.mean())
monthly_riders_since_2008['Total Ridership By Year'] = monthly_riders_since_2008.groupby('Year')['Ridership'].transform(lambda x: x.sum())

#Rearrange columns
monthly_riders_since_2008 = monthly_riders_since_2008.reindex(['Month', 'Year', 'Ridership', 'Mean Ridership By Year', 'Total Ridership By Year'], axis=1)


groued_by_year = monthly_riders_since_2008[['Year', 'Mean Ridership By Year', 'Total Ridership By Year']].drop_duplicates()
groued_by_year



#sns.barplot(data=by_year, x='Year', y='Mean Ridership By Year')
#plt.show()



print(groued_by_year)
monthly_riders_since_2008

     Year  Mean Ridership By Year  Total Ridership By Year
238  2018            1.400050e+08               1680060402
287  2019            1.414823e+08               1697787002
371  2020            5.329509e+07                639541029
455  2021            6.333139e+07                759976721
539  2022            8.445212e+07               1013425465
623  2023            9.582530e+07               1149903621


,Month,Year,Ridership,Mean Ridership By Year,Total Ridership By Year
238,1,2018,134683435,1.400050e+08,1680060402
242,2,2018,127432835,1.400050e+08,1680060402
246,3,2018,143982923,1.400050e+08,1680060402
251,4,2018,141950369,1.400050e+08,1680060402
255,5,2018,150320833,1.400050e+08,1680060402
...,...,...,...,...,...
673,8,2023,94141961,9.582530e+07,1149903621
679,9,2023,95253595,9.582530e+07,1149903621
686,10,2023,103949018,9.582530e+07,1149903621
693,11,2023,98511566,9.582530e+07,1149903621


In [3]:
daily_riders_since_2020 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv', parse_dates=['Date'])

#No missing values
daily_riders_since_2020.dropna(inplace=True)

#Drop unccesary columns that belong to other agencies outside of the subway 
daily_riders_since_2020.drop(daily_riders_since_2020.iloc[:, 3:], inplace=True, axis=1)

#Convert to Datetime
#daily_riders_since_2020['Date'] = pd.to_datetime(daily_riders_since_2020['Date'])

#Break out date into columns for date, month, and year
daily_riders_since_2020['Day'] = daily_riders_since_2020['Date'].dt.day
daily_riders_since_2020['Month'] = daily_riders_since_2020['Date'].dt.month
daily_riders_since_2020['Year'] = daily_riders_since_2020['Date'].dt.year
daily_riders_since_2020['Weekday'] = daily_riders_since_2020['Date'].dt.dayofweek

#Filter out the year 2024
daily_riders_since_2020 = daily_riders_since_2020[daily_riders_since_2020['Year'] < 2024] 

#Sort by date
daily_riders_since_2020.sort_values('Date', ascending=True, inplace=True)




daily_riders_since_2020

,Date,Subways: Total Estimated Ridership,Subways: % of Comparable Pre-Pandemic Day,Day,Month,Year,Weekday
1542,2020-03-02,5329915,0.96,2,3,2020,0
1541,2020-03-03,5481103,0.98,3,3,2020,1
1540,2020-03-04,5498809,0.99,4,3,2020,2
1539,2020-03-05,5496453,0.99,5,3,2020,3
1538,2020-03-06,5189447,0.93,6,3,2020,4
...,...,...,...,...,...,...,...
148,2023-12-27,2912007,0.55,27,12,2023,2
147,2023-12-28,3064841,0.57,28,12,2023,3
146,2023-12-29,3198885,0.60,29,12,2023,4
145,2023-12-30,2440211,0.74,30,12,2023,5


In [4]:
#df3 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', iterator=True, chunksize=1000)
hourly_ridership_since_2022 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', low_memory=False)


#dropping missing values
hourly_ridership_since_2022.dropna(inplace=True)

#Convert timestamp column to Datetime
hourly_ridership_since_2022['transit_timestamp'] = pd.to_datetime(hourly_ridership_since_2022['transit_timestamp'], format='%m/%d/%Y %I:%M:%S %p')

#Breakout timestamp info into diffeent columns
hourly_ridership_since_2022['Day'] = hourly_ridership_since_2022['transit_timestamp'].dt.day
hourly_ridership_since_2022['Month'] = hourly_ridership_since_2022['transit_timestamp'].dt.month
hourly_ridership_since_2022['Year'] = hourly_ridership_since_2022['transit_timestamp'].dt.year
hourly_ridership_since_2022['Day of Week'] = hourly_ridership_since_2022['transit_timestamp'].dt.dayofweek


hourly_ridership_since_2022 = hourly_ridership_since_2022.query('borough != "Staten Island" and transit_mode != "tram" and station_complex_id != "TRAM1"')

#Drop unccesary columns for our analysis
hourly_ridership_since_2022.drop(columns=['payment_method', 'fare_class_category', 'transfers'], inplace=True, axis=1)


#Convert Station ID to int (theres a station id called 'TRAM1'? check if its true)
hourly_ridership_since_2022['station_complex_id'] = hourly_ridership_since_2022['station_complex_id'].astype(int)
    
    
    
hourly_ridership_since_2022

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,ridership,latitude,longitude,Georeference,Day,Month,Year,Day of Week
0,2023-02-12 18:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,84.0,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375),12,2,2023,6
1,2023-03-28 14:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,123.0,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375),28,3,2023,1
2,2023-01-11 02:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,3.0,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375),11,1,2023,2
3,2023-06-08 17:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,200.0,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375),8,6,2023,3
4,2023-07-25 16:00:00,subway,164,"34 St-Penn Station (A,C,E)",Manhattan,872.0,40.752289,-73.993393,POINT (-73.99339294433594 40.752288818359375),25,7,2023,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45109797,2023-08-19 06:00:00,subway,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,1.0,40.749718,-73.987823,POINT (-73.98782348632812 40.749717712402344),19,8,2023,5
45109798,2023-10-06 10:00:00,subway,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,5.0,40.749718,-73.987823,POINT (-73.98782348632812 40.749717712402344),6,10,2023,4
45109799,2023-06-01 16:00:00,subway,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,11.0,40.749718,-73.987823,POINT (-73.98782348632812 40.749717712402344),1,6,2023,3
45109800,2023-05-28 14:00:00,subway,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,1662.0,40.749718,-73.987823,POINT (-73.98782348632812 40.749717712402344),28,5,2023,6


In [5]:
from sqlalchemy import create_engine